In [1]:
!pip install pydantic<2.0, >=1.9
!pip install gradio -q
!pip install pdfminer.six -q
!pip install -U sentence_transformers --q

/bin/bash: line 1: 2.0,: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
import pandas as pd
import gradio as gr
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pdfminer.high_level import extract_text
from sentence_transformers import SentenceTransformer

##SBERT


In [3]:
# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# The App for top CVs


In [4]:
#Extract PDF
def extract_text_from_pdf(file_path):
    return extract_text(file_path)

In [7]:
#Function to compare CVs with job add
def compare(job_ad_file, cv1_file, cv2_file, cv3_file, cv4_file, cv5_file):


    job_ad_text = extract_text_from_pdf(job_ad_file.name)
    cv_text = [extract_text_from_pdf(cv_file.name) for cv_file in [cv1_file, cv2_file, cv3_file, cv4_file, cv5_file]]


    job_ad_embedding = model.encode(job_ad_text)
    cv_embeddings = model.encode(cv_text)

    # Calculate cosine similarity between job ad and each CV
    similarities = cosine_similarity(job_ad_embedding.reshape(1, -1), cv_embeddings)[0]


    results = pd.DataFrame({
            "CV": ["CV 1", "CV 2", "CV 3", "CV 4", "CV 5"],
            "SBERT Similarity": similarities
        })

    return results

In [8]:
#Gradio interface
iface = gr.Interface(fn=compare,
                     inputs=[gr.File(label="Job Advertisement"),
                             gr.File( label="CV 1"),
                             gr.File( label="CV 2"),
                             gr.File( label="CV 3"),
                             gr.File( label="CV 4"),
                             gr.File( label="CV 5")],
                     outputs= gr.DataFrame(),
                     title="Job Ad to CV Matcher",
                     description="Paste one job advertisement and five CVs. The application will provide similarity scores using SBERT.")
iface.launch()

#All 5 CVs must be input for the function to work


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f1ae5ddb489339880c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
